In [238]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import codecs
import pickle
import pandas as pd
from datetime import datetime

In [239]:
results = pd.read_excel('../data/카테고리_분류_2024-09-03.xlsx')
results.head()

,검색어,title,category,멕시코,남미음식,양식,한식,곱창,막창,양,...,인쇄업,스티커인쇄,홍차전문점,크레페,와플,미용,피부,체형관리,차,갤러리카페
0,성수카페거리,마야,멕시코>남미음식,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,성수카페거리,봄의정원 성수점,음식점>양식,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,성수카페거리,사위식당 성수점,"한식>곱창,막창,양",0,0,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
3,성수카페거리,LOOOP 루프 베이커리 카페,"카페,디저트>카페",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,성수카페거리,뚝도농원 성수본점,"한식>육류,고기요리",0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [240]:
review_restaurant = pd.read_excel('../data/음식점_네이버_리뷰_24-09-02_1.xlsx')
review_restaurant['업종'] = '음식점'
review_hotel = pd.read_excel('../data/숙박시설_네이버_리뷰_24-09-02.xlsx')
review_hotel['업종'] = '숙박시설'
review_cafe = pd.read_excel('../data/카페_네이버_리뷰_24-09-02.xlsx')
review_cafe['업종'] = '카페'
reviews = pd.concat([review_restaurant, review_hotel, review_cafe])
reviews.head()

,검색어,검색결과,review,업종
0,성수카페거리,마야,분위기 너무 좋고 맛있었던 성수맛집 마야에서 2인 스페어립 세트 에 고구마 과카몰리...,음식점
1,성수카페거리,마야,서울 한복판에서 마야문명이라니 컨셉하며 인테리어 분위기 음식까지 역대급 핫플 멕시코...,음식점
2,성수카페거리,마야,성수맛집 골라서 찾아다니는데 특별한 인테리어가 돋보였던 마야에서 타코먹었는데 또띠야...,음식점
3,성수카페거리,마야,이름에 걸맞는 마야문명에 온듯한 인테리어 동굴과 모닥불 느낌이 너무 멋지더라구요 특...,음식점
4,성수카페거리,마야,여기딘짜 개존맛도리에요 모르는 사람 없어야되는 맛집곧 줄서서 먹어야될 것 같으니 빠...,음식점


In [241]:
reviews[reviews['검색결과'] == '성수다락']

,검색어,검색결과,review,업종


In [242]:
reviews['review'] = reviews['review'].astype(str)
reviews.rename(columns={'검색결과':'title'}, inplace=True)
reviews

,검색어,title,review,업종
0,성수카페거리,마야,분위기 너무 좋고 맛있었던 성수맛집 마야에서 2인 스페어립 세트 에 고구마 과카몰리...,음식점
1,성수카페거리,마야,서울 한복판에서 마야문명이라니 컨셉하며 인테리어 분위기 음식까지 역대급 핫플 멕시코...,음식점
2,성수카페거리,마야,성수맛집 골라서 찾아다니는데 특별한 인테리어가 돋보였던 마야에서 타코먹었는데 또띠야...,음식점
3,성수카페거리,마야,이름에 걸맞는 마야문명에 온듯한 인테리어 동굴과 모닥불 느낌이 너무 멋지더라구요 특...,음식점
4,성수카페거리,마야,여기딘짜 개존맛도리에요 모르는 사람 없어야되는 맛집곧 줄서서 먹어야될 것 같으니 빠...,음식점
...,...,...,...,...
26058,서울대공원,비밀 베이커리 군자점,샌위치와 커피 모닝빵은 시그니쳐,카페
26059,서울대공원,비밀 베이커리 군자점,일요일 아침 닭가슴살 샐러드가 진리,카페
26060,서울대공원,비밀 베이커리 군자점,버터프레첼이랑 치아바타 프레첼 진짜 쫜득하니 밀도높은 맛 좋아요,카페
26061,서울대공원,비밀 베이커리 군자점,자꾸 생각나요,카페


In [243]:
# 그룹화 후 transform을 사용해 업종 유지
reviews['review_combined'] = reviews.groupby('title')['review'].transform(' '.join)

# 중복된 행 제거
merged_reviews = reviews.drop_duplicates(subset=['title']).reset_index(drop=True)

# 필요한 컬럼만 선택
merged_reviews = merged_reviews[['title', '업종', 'review_combined']].rename(columns={'review_combined': 'review'})

merged_reviews

,title,업종,review
0,마야,음식점,분위기 너무 좋고 맛있었던 성수맛집 마야에서 2인 스페어립 세트 에 고구마 과카몰리...
1,봄의정원 성수점,음식점,예약하고 직장동료분들과 방문했는데 분위기가 좋았어요 추천합니다 특히 머쉬룸크림파스타...
2,사위식당 성수점,음식점,자극적이지 않아서 너무 맛있는 낙곱새에요재료도 듬뿍 들어있고 면사리 추가는 꼭 하세...
3,LOOOP 루프 베이커리 카페,음식점,빵도 맛있고 공간도 다양해서 좋은 카페였습니다 나만 알고 싶은 소중한 카페 평소 ...
4,뚝도농원 성수본점,음식점,성수에서 애정하는 곳 중 하나이만큼 오리구이 잘 하는 집 없음 항상 오리 반마리 먹...
...,...,...,...
384,청주명문도너츠,카페,도넛 금방 만들어서 따끈따끈 맛있어요 역시 명불허전그동안 기름 쩐내나고 뻑뻑하고 푸...
385,IWLT SEOUL,카페,Love the vibes and the peaceful atmosphere Lat...
386,스타벅스 어린이대공원역점,카페,친절하고 맛있고 이용하기 편하고 좋아요 이용하기 편하고 좋아요 오늘 이 동네 터는 ...
387,투썸플레이스 세종대학교점,카페,카페가 넓어서 좋아요 직원분도 친절하시고 음료도 생각한것보다 맛있네요 좋아요 떠먹는...


In [244]:
final_reviews = pd.merge(merged_reviews, results, on='title', how='left')
final_reviews.head()

,title,업종,review,검색어,category,멕시코,남미음식,양식,한식,곱창,...,인쇄업,스티커인쇄,홍차전문점,크레페,와플,미용,피부,체형관리,차,갤러리카페
0,마야,음식점,분위기 너무 좋고 맛있었던 성수맛집 마야에서 2인 스페어립 세트 에 고구마 과카몰리...,성수카페거리,멕시코>남미음식,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,봄의정원 성수점,음식점,예약하고 직장동료분들과 방문했는데 분위기가 좋았어요 추천합니다 특히 머쉬룸크림파스타...,성수카페거리,음식점>양식,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,사위식당 성수점,음식점,자극적이지 않아서 너무 맛있는 낙곱새에요재료도 듬뿍 들어있고 면사리 추가는 꼭 하세...,성수카페거리,"한식>곱창,막창,양",0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,LOOOP 루프 베이커리 카페,음식점,빵도 맛있고 공간도 다양해서 좋은 카페였습니다 나만 알고 싶은 소중한 카페 평소 ...,성수카페거리,"카페,디저트>카페",0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,LOOOP 루프 베이커리 카페,음식점,빵도 맛있고 공간도 다양해서 좋은 카페였습니다 나만 알고 싶은 소중한 카페 평소 ...,성수카페거리,"카페,디저트>카페",0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [247]:
today_date = datetime.now().strftime('%Y-%m-%d')
filename = f'../data/카테고리_분류_리뷰추가_{today_date}.xlsx'
final_reviews.to_excel(filename, index=False)